# Essai automatique

In [58]:
#Temps et fichiers
import os
import warnings
import time
from datetime import timedelta

#Manipulation de données
import pandas as pd
import numpy as np
from pandas_profiling import ProfileReport
from functools import partial


#Modélisation
from sklearn.datasets import fetch_openml
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import PoissonRegressor, GammaRegressor
from sklearn.linear_model import TweedieRegressor
from sklearn.metrics import mean_tweedie_deviance
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder
from sklearn.preprocessing import StandardScaler, KBinsDiscretizer, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error, mean_squared_error, auc

from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import RandomForestClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.svm import LinearSVC
from sklearn.model_selection import RandomizedSearchCV# the keys can be accessed with final_pipeline.get_params().keys()
from sklearn.linear_model import LogisticRegression

from xgboost import XGBClassifier

#Text
import re

#Evaluation
from sklearn.metrics import f1_score, confusion_matrix


#Visualisation
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


#Tracking d'expérience
import mlflow
import mlflow.sklearn

## Chargement des données

In [59]:
random_state=42

In [60]:
df_merged = pd.read_parquet('/mnt/data/interim/df_merged.gzip')

On vire `NBSIN_TYPE1_AN2` => vu avec André : pb dans la variable qui est >=1

In [61]:
df_merged=df_merged.drop('NBSIN_TYPE1_AN2', axis=1)

In [62]:
df_train, df_val = train_test_split(df_merged, shuffle=False, test_size=0.2, random_state=random_state)

In [63]:
X_test=expo_test = pd.read_csv('/mnt/data/raw/expo_test.csv', encoding='utf8', sep=',' )

In [64]:
X_test=X_test.drop(['Unnamed: 0', 'NBSIN_TYPE1_AN2'], axis=1)

In [65]:
X_test

,EXPO,FORMULE,TYPE_RESIDENCE,TYPE_HABITATION,NB_PIECES,SITUATION_JURIDIQUE,NIVEAU_JURIDIQUE,VALEUR_DES_BIENS,OBJETS_DE_VALEUR,ZONIER,NBSIN_TYPE1_AN1,NBSIN_TYPE1_AN3,NBSIN_TYPE2_AN1,NBSIN_TYPE2_AN2,NBSIN_TYPE2_AN3,id,ANNEE
0,1.000000,CONFORT,PRINCIPALE,APPARTEMENT,4.0,LOCATAIRE,JUR1,35000.0,NIVEAU_1,B43,0,0,0,NaN,0,2,2019
1,0.468493,CONFORT,SECONDAIRE,MAISON,1.0,PROPRIO,JUR1,0.0,NIVEAU_1,C22,0,0,0,0.0,0,10,2019
2,0.852055,ESSENTIEL,PRINCIPALE,APPARTEMENT,0.0,PROPRIO,JUR1,0.0,NIVEAU_1,B32,0,0,0,0.0,0,18,2019
3,1.000000,CONFORT,PRINCIPALE,APPARTEMENT,3.0,LOCATAIRE,JUR1,50000.0,NIVEAU_1,C6,0,1,0,0.0,0,19,2019
4,0.079452,CONFORT,SECONDAIRE,MAISON,1.0,PROPRIO,JUR1,9000.0,NIVEAU_1,C6,0,0,0,0.0,0,39,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44344,1.000000,CONFORT,PRINCIPALE,APPARTEMENT,3.0,LOCATAIRE,JUR1,35000.0,NIVEAU_1,C5,0,0,0,0.0,0,499935,2019
44345,0.073973,CONFORT,SECONDAIRE,MAISON,2.0,LOCATAIRE,JUR1,0.0,NIVEAU_1,C6,0,0,0,0.0,0,499953,2019
44346,1.000000,ESSENTIEL,PRINCIPALE,MAISON,2.0,LOCATAIRE,JUR1,20000.0,NIVEAU_1,C13,0,0,0,0.0,0,499964,2019
44347,1.000000,CONFORT,PRINCIPALE,MAISON,2.0,LOCATAIRE,JUR1,9000.0,NIVEAU_1,C17,0,0,0,0.0,0,499969,2019


In [66]:
list(X_test)

['EXPO',
 'FORMULE',
 'TYPE_RESIDENCE',
 'TYPE_HABITATION',
 'NB_PIECES',
 'SITUATION_JURIDIQUE',
 'NIVEAU_JURIDIQUE',
 'VALEUR_DES_BIENS',
 'OBJETS_DE_VALEUR',
 'ZONIER',
 'NBSIN_TYPE1_AN1',
 'NBSIN_TYPE1_AN3',
 'NBSIN_TYPE2_AN1',
 'NBSIN_TYPE2_AN2',
 'NBSIN_TYPE2_AN3',
 'id',
 'ANNEE']

In [67]:
list(df_train)

['EXPO',
 'FORMULE',
 'TYPE_RESIDENCE',
 'TYPE_HABITATION',
 'NB_PIECES',
 'SITUATION_JURIDIQUE',
 'NIVEAU_JURIDIQUE',
 'VALEUR_DES_BIENS',
 'OBJETS_DE_VALEUR',
 'ZONIER',
 'NBSIN_TYPE1_AN1',
 'NBSIN_TYPE1_AN3',
 'NBSIN_TYPE2_AN1',
 'NBSIN_TYPE2_AN2',
 'NBSIN_TYPE2_AN3',
 'id',
 'ANNEE',
 'NB',
 'COUT',
 'PurePremium',
 'Frequency',
 'AvgClaimAmount']

In [68]:
y_train = df_train[['id','EXPO','NB',
 'COUT',
 'PurePremium',
 'Frequency',
 'AvgClaimAmount']]

In [69]:
y_train

,id,EXPO,NB,COUT,PurePremium,Frequency,AvgClaimAmount
0,5,1.000000,0.0,0.0,0.0,0.0,0.0
1,9,0.824657,0.0,0.0,0.0,0.0,0.0
2,11,1.000000,0.0,0.0,0.0,0.0,0.0
3,13,1.000000,0.0,0.0,0.0,0.0,0.0
4,14,1.000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
124515,400345,1.000000,0.0,0.0,0.0,0.0,0.0
124516,400348,1.000000,0.0,0.0,0.0,0.0,0.0
124517,400350,0.273973,0.0,0.0,0.0,0.0,0.0
124518,400356,1.000000,0.0,0.0,0.0,0.0,0.0


In [70]:
y_train.describe()

,id,EXPO,NB,COUT,PurePremium,Frequency,AvgClaimAmount
count,124520.000000,124520.000000,124520.000000,124520.000000,124520.000000,124520.000000,124520.000000
mean,200071.902337,0.837586,0.017917,24.986664,32.123230,0.022580,24.359367
std,115728.477177,0.296708,0.136705,376.735988,749.479255,0.281951,369.059132
min,5.000000,0.002732,0.000000,0.000000,0.000000,0.000000,0.000000
25%,99973.500000,0.816438,0.000000,0.000000,0.000000,0.000000,0.000000
50%,200222.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,300233.500000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,400357.000000,1.000000,3.000000,33089.120000,185244.917127,52.285749,33089.120000


In [71]:
X_train = df_train.drop(['NB',
 'COUT',
 'PurePremium',
 'Frequency',
 'AvgClaimAmount'], axis=1)

In [72]:
X_train

,EXPO,FORMULE,TYPE_RESIDENCE,TYPE_HABITATION,NB_PIECES,SITUATION_JURIDIQUE,NIVEAU_JURIDIQUE,VALEUR_DES_BIENS,OBJETS_DE_VALEUR,ZONIER,NBSIN_TYPE1_AN1,NBSIN_TYPE1_AN3,NBSIN_TYPE2_AN1,NBSIN_TYPE2_AN2,NBSIN_TYPE2_AN3,id,ANNEE
0,1.000000,MEDIUM,PRINCIPALE,APPARTEMENT,1.0,PROPRIO,JUR1,3500.0,NIVEAU_1,B40,0,0,0,0.0,0,5,2017
1,0.824657,CONFORT,PRINCIPALE,MAISON,NaN,PROPRIO,JUR1,0.0,NIVEAU_1,A11,0,0,0,0.0,0,9,2018
2,1.000000,ESSENTIEL,PRINCIPALE,APPARTEMENT,3.0,LOCATAIRE,JUR1,35000.0,NIVEAU_1,B32,0,0,0,0.0,0,11,2017
3,1.000000,ESSENTIEL,SECONDAIRE,MAISON,2.0,LOCATAIRE,JUR1,9000.0,NIVEAU_1,C24,0,0,0,0.0,0,13,2018
4,1.000000,ESSENTIEL,PRINCIPALE,MAISON,1.0,LOCATAIRE,JUR1,20000.0,NIVEAU_1,C9,0,0,0,0.0,0,14,2017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124515,1.000000,CONFORT,PRINCIPALE,MAISON,1.0,PROPRIO,JUR1,3500.0,NIVEAU_1,C20,1,0,0,0.0,0,400345,2018
124516,1.000000,MEDIUM,PRINCIPALE,MAISON,2.0,PROPRIO,JUR1,3500.0,NIVEAU_1,A2,0,0,0,0.0,0,400348,2017
124517,0.273973,CONFORT,SECONDAIRE,MAISON,1.0,PROPRIO,JUR1,3500.0,NIVEAU_1,C16,0,0,0,0.0,0,400350,2018
124518,1.000000,MEDIUM,PRINCIPALE,MAISON,2.0,LOCATAIRE,JUR1,3500.0,NIVEAU_1,C9,0,0,0,0.0,0,400356,2016


In [73]:
X_val = df_val.drop(['NB',
 'COUT',
 'PurePremium',
 'Frequency',
 'AvgClaimAmount'], axis=1)

In [74]:
y_val = df_val[['id','EXPO','NB',
 'COUT',
 'PurePremium',
 'Frequency',
 'AvgClaimAmount']]

In [75]:
y_val

,id,EXPO,NB,COUT,PurePremium,Frequency,AvgClaimAmount
124520,400364,0.775342,0.0,0.0,0.0,0.0,0.0
124521,400365,1.000000,0.0,0.0,0.0,0.0,0.0
124522,400366,1.000000,0.0,0.0,0.0,0.0,0.0
124523,400367,1.000000,0.0,0.0,0.0,0.0,0.0
124524,400368,1.000000,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
155646,499986,0.680327,0.0,0.0,0.0,0.0,0.0
155647,499989,1.000000,0.0,0.0,0.0,0.0,0.0
155648,499991,0.569863,0.0,0.0,0.0,0.0,0.0
155649,499993,0.597260,0.0,0.0,0.0,0.0,0.0


## Essai opti

In [76]:
from supervised.automl import AutoML # mljar-supervised

# train models with AutoML
automl = AutoML(mode="Explain", results_path='/mnt/auto_ml/mljar-ChargePos')
automl.fit(X_train[y_train['NB']>0].drop('id',axis=1), y_train[y_train['NB']>0]['COUT'])

# compute the MSE on test data
predictions = automl.predict(X_val[y_val['NB']>0].drop('id',axis=1))
print("Test MSE:", mean_squared_error(y_val[y_val['NB']>0]['COUT'], predictions))

This model has already been fitted. You can use predict methods or select a new 'results_path' for a new 'fit()'.
Test MSE: 4772314.4918454755


In [77]:
from supervised.automl import AutoML # mljar-supervised

# train models with AutoML
automl = AutoML(mode="Explain", results_path='/mnt/auto_ml/mljar-HasSin')
automl.fit(X_train.drop('id',axis=1), y_train['NB'].apply(lambda x : min(1,x)))

# compute the MSE on test data
predictions = automl.predict(X_val.drop('id',axis=1))
print("Test MSE:", mean_squared_error(y_val['NB'].apply(lambda x : min(1,x)), predictions))

Linear algorithm was disabled.
This model has already been fitted. You can use predict methods or select a new 'results_path' for a new 'fit()'.
Test MSE: 0.02354566188044072


In [78]:
y_train['a_sinistre']=y_train['NB']
y_train['a_sinistre']=y_train['a_sinistre'].apply(lambda x : 1 if x>0 else 0)
y_train.describe()
#.apply(lambda x : 1 if x>0 else 0)

,id,EXPO,NB,COUT,PurePremium,Frequency,AvgClaimAmount,a_sinistre
count,124520.000000,124520.000000,124520.000000,124520.000000,124520.000000,124520.000000,124520.000000,124520.000000
mean,200071.902337,0.837586,0.017917,24.986664,32.123230,0.022580,24.359367,0.017379
std,115728.477177,0.296708,0.136705,376.735988,749.479255,0.281951,369.059132,0.130678
min,5.000000,0.002732,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,99973.500000,0.816438,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,200222.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,300233.500000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,400357.000000,1.000000,3.000000,33089.120000,185244.917127,52.285749,33089.120000,1.000000


In [79]:
y_val['a_sinistre']=y_val['NB']
y_val['a_sinistre']=y_val['a_sinistre'].apply(lambda x : 1 if x>0 else 0)
y_val.describe()
#.apply(lambda x : 1 if x>0 else 0)

,id,EXPO,NB,COUT,PurePremium,Frequency,AvgClaimAmount,a_sinistre
count,31131.000000,31131.000000,31131.000000,31131.000000,31131.000000,31131.000000,31131.000000,31131.000000
mean,450324.038932,0.836046,0.017442,23.074944,33.492690,0.022573,22.549844,0.016993
std,28783.886800,0.297231,0.134306,334.822067,726.362066,0.283973,326.348253,0.129246
min,400364.000000,0.002732,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,425450.000000,0.797260,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,450056.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,475468.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,499999.000000,1.000000,2.000000,18237.690000,80150.093324,33.272745,18237.690000,1.000000


# A la mano

In [80]:
# Define categorical pipeline
cat_pipe = Pipeline([('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
                     ('encoder', OneHotEncoder(handle_unknown='ignore', sparse=False))])

# Define numerical pipeline
num_pipe = Pipeline([('imputer', SimpleImputer(strategy='median')),
                     ('scaler', MinMaxScaler())])

num_pipe_binned = Pipeline([('imputer', SimpleImputer(strategy='median')),
                     ('scaler', KBinsDiscretizer(n_bins=10))])

num_pipe_log = Pipeline([('imputer', SimpleImputer(strategy='median')),
                     ('scaler', KBinsDiscretizer(n_bins=10))])


# Fit column transformer to training data
preprocessor = ColumnTransformer(transformers=[('cat', cat_pipe, ["FORMULE", "TYPE_RESIDENCE", "SITUATION_JURIDIQUE",'NIVEAU_JURIDIQUE','OBJETS_DE_VALEUR', 'ZONIER', 'NBSIN_TYPE1_AN1', 'NBSIN_TYPE1_AN3', 'NBSIN_TYPE2_AN1',  'NBSIN_TYPE2_AN3']),
                                               ('num_binned', num_pipe_binned, ["VALEUR_DES_BIENS"])],remainder='drop')

In [93]:
X_train_transf=preprocessor.fit_transform(X_train)

In [99]:
model = RandomForestClassifier(n_estimators=50, max_depth=5,class_weight="balanced")
model.fit(X_train_transf, y_train['a_sinistre'])

RandomForestClassifier(class_weight='balanced', max_depth=5, n_estimators=50)

In [100]:
X_val_transf=preprocessor.transform(X_val)
y_pred = model.predict(X_val_transf)
print(classification_report(y_val['a_sinistre'], y_pred))

              precision    recall  f1-score   support

           0       0.99      0.57      0.72     30602
           1       0.03      0.66      0.05       529

    accuracy                           0.57     31131
   macro avg       0.51      0.61      0.38     31131
weighted avg       0.97      0.57      0.71     31131



In [101]:
!pip install -U imbalanced-learn

In [104]:
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.tree import DecisionTreeClassifier

In [105]:
bbc = BalancedBaggingClassifier(base_estimator=DecisionTreeClassifier(),
                                 sampling_strategy='auto',
                                replacement=False,
                                random_state=0)

In [106]:
model = bbc
model.fit(X_train_transf, y_train['a_sinistre'])

BalancedBaggingClassifier(base_estimator=DecisionTreeClassifier(),
                          random_state=0)

In [107]:
X_val_transf=preprocessor.transform(X_val)
y_pred = model.predict(X_val_transf)
print(classification_report(y_val['a_sinistre'], y_pred))

              precision    recall  f1-score   support

           0       0.99      0.63      0.77     30602
           1       0.02      0.55      0.05       529

    accuracy                           0.62     31131
   macro avg       0.51      0.59      0.41     31131
weighted avg       0.97      0.62      0.75     31131



In [108]:
from imblearn.ensemble import BalancedRandomForestClassifier

In [109]:
model = BalancedRandomForestClassifier(n_estimators=100, random_state=0)
model.fit(X_train_transf, y_train['a_sinistre'])

BalancedRandomForestClassifier(random_state=0)

In [110]:
X_val_transf=preprocessor.transform(X_val)
y_pred = model.predict(X_val_transf)
print(classification_report(y_val['a_sinistre'], y_pred))

              precision    recall  f1-score   support

           0       0.99      0.52      0.68     30602
           1       0.02      0.70      0.05       529

    accuracy                           0.52     31131
   macro avg       0.51      0.61      0.36     31131
weighted avg       0.97      0.52      0.67     31131



In [111]:
from imblearn.ensemble import RUSBoostClassifier

In [112]:
model = RUSBoostClassifier(n_estimators=200, algorithm='SAMME.R',random_state=0)
model.fit(X_train_transf, y_train['a_sinistre'])

RUSBoostClassifier(n_estimators=200, random_state=0)

In [113]:
X_val_transf=preprocessor.transform(X_val)
y_pred = model.predict(X_val_transf)
print(classification_report(y_val['a_sinistre'], y_pred))

              precision    recall  f1-score   support

           0       0.99      0.59      0.74     30602
           1       0.03      0.66      0.05       529

    accuracy                           0.59     31131
   macro avg       0.51      0.62      0.40     31131
weighted avg       0.97      0.59      0.73     31131



In [115]:
from imblearn.metrics import classification_report_imbalanced

In [116]:
print(classification_report_imbalanced(y_val['a_sinistre'], y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.99      0.59      0.66      0.74      0.62      0.38     30602
          1       0.03      0.66      0.59      0.05      0.62      0.39       529

avg / total       0.97      0.59      0.65      0.73      0.62      0.38     31131



In [87]:
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from imblearn.under_sampling import EditedNearestNeighbours
from imblearn.over_sampling import SMOTE

pca = PCA(n_components=2)
enn = EditedNearestNeighbours()
smote = SMOTE(random_state=0)
knn = KNeighborsClassifier(n_neighbors=1)

In [88]:
from imblearn.pipeline import make_pipeline

model = make_pipeline(pca, enn, smote, knn)

In [92]:
from sklearn.metrics import classification_report

model.fit(X_train_transf, y_train['a_sinistre'])
y_pred = model.predict(X_val_transf)
print(classification_report(y_val['a_sinistre'], y_pred))

              precision    recall  f1-score   support

           0       0.98      0.79      0.88     30602
           1       0.02      0.30      0.05       529

    accuracy                           0.78     31131
   macro avg       0.50      0.55      0.46     31131
weighted avg       0.97      0.78      0.86     31131



In [91]:
y_pred

array([0, 0, 0, ..., 0, 1, 1])